In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import transforms, datasets

# CNN을 통해 이미지를 학습시켜보자!!
CV기술의 기본이 되는 Convolution의 개념을 알고 이미지 분류를 위한 인공신경망을 만들어보자
---

In [2]:
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [3]:
#하이퍼 파라미터 설정
EPOCHS = 40
BATCH_SIZE = 64

In [4]:
#데이터 불러오기
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train = True,
                   download=True,
                   transform =transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,),(0.3081,))
                   ])),
    batch_size = BATCH_SIZE, shuffle=True)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('./.data',
                   train=False,
                   transform = transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,),(0.3081,))
                   ])),
    batch_size = BATCH_SIZE, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./.data\MNIST\raw\train-images-idx3-ubyte.gz to ./.data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./.data\MNIST\raw\train-labels-idx1-ubyte.gz to ./.data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./.data\MNIST\raw\t10k-images-idx3-ubyte.gz to ./.data\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./.data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ./.data\MNIST\raw



In [5]:
#모델 선언하기
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,10,kernel_size=5)#1채널 이미지를 10채널 특징맵으로... 5x5 커널을 이용해 만들기!!
        self.conv2 = nn.Conv2d(10,20,kernel_size=5)#10채널을 20채널로..
        self.conv2_drop = nn.Dropout2d() #드롭아웃 선언하고
        self.fc1 = nn.Linear(320,50) #입력값 고려해서 레이어 만들기
        self.fc2 = nn.Linear(50,10) #출력값 고려해서 마지막 레이어 만들기
    def forward(self,x):
        x = F.relu(F.max_pool2d(self.conv1(x),2)) #conv1 이후 맥스 풀링 커널 사이즈는 2x2, 엑티베이션은 렐루!
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)),2))
        x = x.view(-1,320) #텐서 펴주기
        x = F.relu(self.fc1(x)) 
        x = F.dropout(x,training=self.training) #드롭아웃하기
        x = self.fc2(x)
        return F.log_softmax(x,dim=1) #소프트맥스를 거쳐 반환

In [6]:
model = Net().to(DEVICE) #모델 생성하고
optimizer = optim.SGD(model.parameters(),lr=0.01,momentum=0.5) #하이퍼 파라미터 정하기

In [7]:
#모델을 학습 시키는 데피니션
def train(model, train_loader,optimizer,epoch):
    model.train()
    for batch_idx, (data,target) in enumerate(train_loader):
        data, target = data.to(DEVICE),target.to(DEVICE)
        optimizer.zero_grad()
        output = model(data)
        loss = F.cross_entropy(output,target)
        loss.backward()
        optimizer.step()
        
        #학습이 얼마나 되었는지, 오차는 얼마인지 출력하는 코드
        if batch_idx%200==0:
            print('Train Epoch: {}[{}/{} ({:.0f}%)]\tloss:{:.6f}'.format(
                epoch,batch_idx*len(data),len(train_loader.dataset),100.*batch_idx/len(train_loader),loss.item()))

In [8]:
#모델의 성능을 검사하는 데피니션
def evaluate(model, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data,target in test_loader:
            data, target = data.to(DEVICE), target.to(DEVICE)
            output = model(data)
            
            test_loss +=F.cross_entropy(output,target,reduction='sum').item()
            
            pred = output.max(1,keepdim=True)[1]
            correct += pred.eq(target.view_as(pred)).sum().item()
            
    test_loss /= len(test_loader.dataset)
    test_accuracy = 100.*correct/len(test_loader.dataset)
    return test_loss, test_accuracy

In [9]:
#학습한 후 성능 검사하기!!
for epoch in range(1, EPOCHS+1):
    train(model, train_loader,optimizer,epoch)
    test_loss,test_accuracy = evaluate(model,test_loader)
    
    print('[{}] Test Loss: {:.4f}, Accuracy: {:2f}%'.format(epoch, test_loss, test_accuracy))

Train Epoch: 1[0/60000 (0%)]	loss:2.384162
Train Epoch: 1[12800/60000 (21%)]	loss:1.225545
Train Epoch: 1[25600/60000 (43%)]	loss:0.614739
Train Epoch: 1[38400/60000 (64%)]	loss:0.551042
Train Epoch: 1[51200/60000 (85%)]	loss:0.337811
[1] Test Loss: 0.1862, Accuracy: 94.590000%
Train Epoch: 2[0/60000 (0%)]	loss:0.340572
Train Epoch: 2[12800/60000 (21%)]	loss:0.567270
Train Epoch: 2[25600/60000 (43%)]	loss:0.302098
Train Epoch: 2[38400/60000 (64%)]	loss:0.271652
Train Epoch: 2[51200/60000 (85%)]	loss:0.252258
[2] Test Loss: 0.1256, Accuracy: 96.160000%
Train Epoch: 3[0/60000 (0%)]	loss:0.468850
Train Epoch: 3[12800/60000 (21%)]	loss:0.201984
Train Epoch: 3[25600/60000 (43%)]	loss:0.317321
Train Epoch: 3[38400/60000 (64%)]	loss:0.322195
Train Epoch: 3[51200/60000 (85%)]	loss:0.227535
[3] Test Loss: 0.0928, Accuracy: 97.130000%
Train Epoch: 4[0/60000 (0%)]	loss:0.223916
Train Epoch: 4[12800/60000 (21%)]	loss:0.149008
Train Epoch: 4[25600/60000 (43%)]	loss:0.190800
Train Epoch: 4[38400/600

Train Epoch: 30[12800/60000 (21%)]	loss:0.150374
Train Epoch: 30[25600/60000 (43%)]	loss:0.032207
Train Epoch: 30[38400/60000 (64%)]	loss:0.049023
Train Epoch: 30[51200/60000 (85%)]	loss:0.109272
[30] Test Loss: 0.0346, Accuracy: 98.880000%
Train Epoch: 31[0/60000 (0%)]	loss:0.126692
Train Epoch: 31[12800/60000 (21%)]	loss:0.125481
Train Epoch: 31[25600/60000 (43%)]	loss:0.128112
Train Epoch: 31[38400/60000 (64%)]	loss:0.059534
Train Epoch: 31[51200/60000 (85%)]	loss:0.049484
[31] Test Loss: 0.0343, Accuracy: 98.940000%
Train Epoch: 32[0/60000 (0%)]	loss:0.056619
Train Epoch: 32[12800/60000 (21%)]	loss:0.022872
Train Epoch: 32[25600/60000 (43%)]	loss:0.050299
Train Epoch: 32[38400/60000 (64%)]	loss:0.101238
Train Epoch: 32[51200/60000 (85%)]	loss:0.033201
[32] Test Loss: 0.0320, Accuracy: 98.980000%
Train Epoch: 33[0/60000 (0%)]	loss:0.105006
Train Epoch: 33[12800/60000 (21%)]	loss:0.109898
Train Epoch: 33[25600/60000 (43%)]	loss:0.197904
Train Epoch: 33[38400/60000 (64%)]	loss:0.07678

DNN으로 돌렸을땐 약 97.3%의 정확도를 가지지만 CNN은 98.9%의 정확도를 가진다.
---